In [ ]:
import json
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch
import matplotlib.pyplot as plt

# Load the configuration file
with open('path_to_config.json', 'r') as config_file:
    config = json.load(config_file)

# Paths
base_dir = config['paths']['base_dir']
csv_path = config['paths']['csv_path']
img_dir = config['paths']['img_dir']
model_save_path = config['paths']['model_save_path']

# Training parameters
validation_split = config['training']['validation_split']
fraction = config['training']['fraction']
batch_size = config['training']['batch_size']
epochs = config['training']['epochs']
initial_learning_rate = config['training']['initial_learning_rate']
learning_rate_decay = config['training']['learning_rate_decay']
patience = config['training']['patience']
reduce_lr_factor = config['training']['reduce_lr_factor']
reduce_lr_patience = config['training']['reduce_lr_patience']
min_lr = config['training']['min_lr']

# Data augmentation parameters
data_aug_params = config['data_augmentation']

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=data_aug_params['rescale'],
    rotation_range=data_aug_params['rotation_range'],
    width_shift_range=data_aug_params['width_shift_range'],
    height_shift_range=data_aug_params['height_shift_range'],
    shear_range=data_aug_params['shear_range'],
    zoom_range=data_aug_params['zoom_range'],
    horizontal_flip=data_aug_params['horizontal_flip'],
    vertical_flip=data_aug_params['vertical_flip'],
    brightness_range=data_aug_params['brightness_range'],
    validation_split=validation_split
)

train_generator = train_datagen.flow_from_directory(
    img_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    img_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

class ResNet50HyperModel(HyperModel):
    def build(self, hp):
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        for layer in base_model.layers:
            layer.trainable = False
        
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(hp.Int('units', min_value=128, max_value=1024, step=128), activation='relu')(x)
        x = Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.7, step=0.1))(x)
        outputs = Dense(1, activation='sigmoid')(x)
        
        model = Model(inputs=base_model.input, outputs=outputs)
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default=initial_learning_rate)),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        return model

# Hyperparameter tuner
tuner = RandomSearch(
    ResNet50HyperModel(),
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=1,
    directory=base_dir,
    project_name='skin_lesion_tuning'
)

# Callbacks
early_stopping = EarlyStopping(patience=patience, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=reduce_lr_factor, patience=reduce_lr_patience, min_lr=min_lr)

# Run the tuner
tuner.search(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the best model
model = tuner.hypermodel.build(best_hps)

# Train the model with the best hyperparameters
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)

# Save the model
model.save(model_save_path)

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()